<a href="https://colab.research.google.com/github/SadaouiDyhia/Simulated-annealing-for-TSP/blob/main/SimulatedAnnealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import sin, cos,acos, sqrt, atan2, radians, exp
import random
import matplotlib.pyplot as plt
import time
import numpy as np

liste=[]
#Visualisation 
def plotTSP(paths, points, num_iters=1):
    # Unpack the primary TSP path and transform it into a list of ordered
    # coordinates

    x = []; y = []
    for i in paths[0]:
        x.append(points[i][0])
        y.append(points[i][1])

    plt.plot(x, y, 'co')

    # Set a scale for the arrow heads (there should be a reasonable default for this, WTF?)
    a_scale = float(max(x))/float(100)

    # Draw the older paths, if provided
    if num_iters > 1:

        for i in range(1, num_iters):

            # Transform the old paths into a list of coordinates
            xi = []; yi = [];
            for j in paths[i]:
                xi.append(points[j][0])
                yi.append(points[j][1])

            plt.arrow(xi[-1], yi[-1], (xi[0] - xi[-1]), (yi[0] - yi[-1]),
                    head_width = a_scale, color = 'r',
                    length_includes_head = True, ls = 'dashed',
                    width = 0.001/float(num_iters))
            for i in range(0, len(x) - 1):
                plt.arrow(xi[i], yi[i], (xi[i+1] - xi[i]), (yi[i+1] - yi[i]),
                        head_width = a_scale, color = 'r', length_includes_head = True,
                        ls = 'dashed', width = 0.001/float(num_iters))

    # Draw the primary path for the TSP problem
    plt.arrow(x[-1], y[-1], (x[0] - x[-1]), (y[0] - y[-1]), head_width = a_scale,
            color ='g', length_includes_head=True)
    for i in range(0,len(x)-1):
        plt.arrow(x[i], y[i], (x[i+1] - x[i]), (y[i+1] - y[i]), head_width = a_scale,
                color = 'g', length_includes_head = True)

    #Set axis too slitghtly larger than the set of x and y
    plt.xlim(min(x)*1.1, max(x)*1.1)
    plt.ylim(min(y)*1.1, max(y)*1.1)
    plt.show()



class SimAnneal(object):
    def __init__(self, coords, T=-1, alpha=-1, stopping_T=-1, stopping_iter=-1):
        self.coords = coords
        self.N = len(coords)
        self.T = sqrt(self.N) if T == -1 else T
        self.T_save = self.T  # save inital T to reset if batch annealing is used
        self.alpha = 0.995 if alpha == -1 else alpha
        self.stopping_temperature = 1e-8 if stopping_T == -1 else stopping_T
        self.stopping_iter = 100000 if stopping_iter == -1 else stopping_iter
        self.iteration = 1

        self.nodes = [i for i in range(self.N)]

        self.best_solution = None
        self.best_fitness = float("Inf")
        self.fitness_list = []

    def initial_solution(self):
        cur_node = random.choice(self.nodes)  # start from a random node
        solution = [cur_node]

        free_nodes = set(self.nodes)
        free_nodes.remove(cur_node)
        while free_nodes:
            next_node = min(free_nodes, key=lambda x: self.gdist(cur_node, x))  # nearest neighbour
            free_nodes.remove(next_node)
            solution.append(next_node)
            cur_node = next_node

        cur_fit = self.fitness(solution)
        if cur_fit < self.best_fitness:  # If best found so far, update best fitness
            self.best_fitness = cur_fit
            self.best_solution = solution
        self.fitness_list.append(cur_fit)
        return solution, cur_fit


    def dist(self, node_0, node_1):
        coord_0, coord_1 = self.coords[node_0], self.coords[node_1]
        return math.sqrt((coord_0[0] - coord_1[0]) ** 2 + (coord_0[1] - coord_1[1]) ** 2)
    
    def gdist(self,node_0, node_1):
        Y, X  = self.coords[node_0], self.coords[node_1]
        PI = 3.141592
        deg = int(float(X[0]))
        minu = float(X[0]) - deg
        latitude_x = (PI * (deg + 5.0 * minu / 3.0 )) / 180.0
        deg = int(float(X[1]))
        minu = X[1] - deg
        longitude_x = (PI * (deg + 5.0 * minu / 3.0 )) / 180.0
      
        deg = int(float(Y[0]))
        minu = Y[0] - deg
        latitude_y = (PI * (deg + 5.0 * minu / 3.0 )) / 180.0
        deg = int(float(Y[1]))
        minu = Y[1] - deg
        longitude_y = (PI * (deg + 5.0 * minu / 3.0 )) / 180.0
        
        RRR = 6378.388
        q1 = cos( longitude_x - longitude_y )
        q2 = cos( latitude_x - latitude_y )
        q3 = cos( latitude_x + latitude_y )
        dij = (int) ( RRR * acos( 0.5*((1.0+q1)*q2 - (1.0-q1)*q3) ) + 1.0)
        return dij


    def fitness(self, solution):
        cur_fit = 0
        for i in range(self.N):
            cur_fit += self.gdist(solution[i % self.N], solution[(i + 1) % self.N])
        return cur_fit

    def p_accept(self, candidate_fitness):
        return exp(-abs(candidate_fitness - self.cur_fitness) / self.T)

    def accept(self, candidate):
        candidate_fitness = self.fitness(candidate)
        if candidate_fitness < self.cur_fitness:
            self.cur_fitness, self.cur_solution = candidate_fitness, candidate
            if candidate_fitness < self.best_fitness:
                self.best_fitness, self.best_solution = candidate_fitness, candidate
        else:
            if random.random() < self.p_accept(candidate_fitness):
                self.cur_fitness, self.cur_solution = candidate_fitness, candidate

    def anneal(self):
        # Initialize with the greedy solution.
        self.cur_solution, self.cur_fitness = self.initial_solution()
        print("Solution initiale :", self.cur_fitness )

        print(" Execution en cours ...  \n nb iterations:",self.stopping_iter, "\n init temp:",self.T, "\n stopping temp: ",self.stopping_temperature, "\n alpha:", self.alpha   )
        
        while self.T >= self.stopping_temperature and self.iteration < self.stopping_iter:
            candidate = list(self.cur_solution)
            l = random.randint(2, self.N - 1)
            i = random.randint(0, self.N - l)
            candidate[i : (i + l)] = reversed(candidate[i : (i + l)])
            self.accept(candidate)
            self.T *= self.alpha
            self.iteration += 1
            self.fitness_list.append(self.cur_fitness)
       
        print("Solution optimale approchée avec RS: ", self.best_fitness)
        improvement = 100 * (self.fitness_list[0] - self.best_fitness) / (self.fitness_list[0])
        print(f"Amelioration par rapport a la solution initiale: {improvement : .2f}%")

    def visualize_routes(self):
        plotTSP([self.best_solution], self.coords)

    def plot_learning(self):
        print("length of fitness list", len(self.fitness_list))
        plt.plot([i for i in range(len(self.fitness_list))], self.fitness_list)
        plt.ylabel("Cout")
        plt.xlabel("NumSolutionTrouvée")
        plt.show()
      
    


def test(coords, times, iter ): 
        global liste
        for i in range(1, times + 1):
            print(f"Iteration {i}/{times} -------------------------------")
            sa = SimAnneal(coords,-1,-1,-1, iter)
            sa.anneal()
            if len(liste)==0:
               liste=sa.fitness_list
               
            else:
              liste=[(g + h) / 2 for g, h in zip(liste, sa.fitness_list)]
              


#Reading coords 
def read_coords():
    from google.colab import files
    file = files.upload()
    coords = []
    with open("bays29.txt", "r") as f:
        for line in f.readlines():
            line = [float(x.replace("\n", "")) for x in line.split("  ")]
            coords.append(line)
    return coords

#Generating random coords for tests
def generate_random_coords(num_nodes):
    return [[random.uniform(-1000, 1000), random.uniform(-1000, 1000)] for i in range(num_nodes)]



#########Test
if __name__ == "__main__":
    #read coords 
    coords = read_coords()   # generate_random_coords(100)
    
    #sa = SimAnneal(coords,-1,-1,-1, 1000) 
    global liste
    liste.clear(); 
    t = time.time()
    test(coords,1, 4000)
    t2= time.time()
    time = round(t2 - t, 3)
    print(f"Exec time {time} secondes.") 
    
    
    plt.plot([i for i in range(len(liste))], liste)
    plt.ylabel("Cout")
    plt.xlabel("Iteration ")
    plt.ticklabel_format(useOffset=False)
    plt.show()


    #for visualization
    #sa.visualize_routes()
    #sa.plot_learning()
